# Matrix Multiplication 

In simple terms, mathematically,  

If **A** is an *m × n* matrix and **B** is an *n × p* matrix:

$$
\mathbf{A} = \begin{pmatrix}
 a_{11} & a_{12} & \cdots & a_{1n} \\
 a_{21} & a_{22} & \cdots & a_{2n} \\
 \vdots & \vdots & \ddots & \vdots \\
 a_{m1} & a_{m2} & \cdots & a_{mn} \\
\end{pmatrix},\quad
\mathbf{B} = \begin{pmatrix}
 b_{11} & b_{12} & \cdots & b_{1p} \\
 b_{21} & b_{22} & \cdots & b_{2p} \\
 \vdots & \vdots & \ddots & \vdots \\
 b_{n1} & b_{n2} & \cdots & b_{np} \\
\end{pmatrix}
$$

Then, the matrix product **C = AB**, where **C** is an *m × p* matrix, is defined as:

$$
\mathbf{C} = \begin{pmatrix}
 c_{11} & c_{12} & \cdots & c_{1p} \\
 c_{21} & c_{22} & \cdots & c_{2p} \\
 \vdots & \vdots & \ddots & \vdots \\
 c_{m1} & c_{m2} & \cdots & c_{mp} \\
\end{pmatrix}
$$

where each element is calculated by:

$$
c_{ij} = \sum_{k=1}^{n} a_{ik} \cdot b_{kj}
$$

for all $$ i = 1, 2, \dots, m $$ and $$ j = 1, 2, \dots, p $$.


In [ ]:
#| default_exp matrix_multiply

In [ ]:
#| export
from typing import List, Tuple
import numpy as np
import torch

## Brute Force Methodology

This approach uses the most basic and straightforward method of matrix multiplication, directly applying the definition outlined above.

In [ ]:
#| exports

def matrix_multiply(a: List[List[float]], # input matrix of size (m, n)
                    b: List[List[float]] # input matrix of size (n, p)
                    ) -> List[List[float]]: # output matrix of size (m, p)
    a = np.array(a)
    b = np.array(b)
    ar, ac = a.shape
    br, bc = b.shape
    if ac != br:
        raise ValueError("Incompatible shapes for matrix multiplication")
    result = np.zeros((ar, bc))
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): # or br works too
                result[i][j] += a[i][k] * b[k][j]
    return result.tolist()

## PyTorch Optimization

PyTorch leverages **ATen**, a high-performance tensor library written in C++. At its core, PyTorch optimizes matrix multiplication by utilizing efficient element-wise operations and parallel computation under the hood. These low-level implementations significantly accelerate the computation.  

Let’s now rewrite the above matrix multiplication using PyTorch's element-wise operations to take advantage of this optimization.

In [ ]:
#| exports

def matrix_multiply_torch(a: List[List[float]], # input matrix of size (m, n)
                          b: List[List[float]] # input matrix of size (n, p)
                        ) -> List[List[float]]: # output matrix of size (m, p)
        a = torch.tensor(a)
        b = torch.tensor(b)
        ar, ac = a.shape
        br, bc = b.shape
        if ac != br:
            raise ValueError("Incompatible shapes for matrix multiplication")
        result = torch.zeros((ar, bc))
        for i in range(ar):
            for j in range(bc):
                result[i][j] = torch.sum(a[i, :] * b[:, j])
        return result

## Numpy Broadcasting

Broadcasting is a method used by Pytorch (and libraries like NumPy) to automatically expand the shape of smaller arrays so that arithmetic operations (like addition, multiplication) can happen without explicit looping in Python.

### Step by Step Breakdown

Let's walk through the broadcasting logic:

```python
c[i] = (a[i,:].unsqueeze(-1) * b).sum(dim=0)
```

Let’s assume:
- `a` is of shape `(ar, ac)` => e.g., `(2, 3)`
- `b` is of shape `(br, bc)` => e.g., `(3, 4)`
- So `ac == br` => matrix multiplication is valid
- `c` will have shape `(ar, bc)` => e.g., `(2, 4)`

#### 1. `a[i, :]`
This picks the `i`th row of `a`, which is a 1D tensor of shape `(ac,)` => e.g., shape `(3,)`

#### 2. `.unsqueeze(-1)`
Adds a new axis at the end (i.e., converts a row vector to a column vector). So now:
```python
a[i,:].unsqueeze(-1) =>  shape becomes (ac, 1) => e.g., (3, 1)
```

#### 3. `a[i,:].unsqueeze(-1) * b`
Here’s where **broadcasting** happens.

- `a[i,:].unsqueeze(-1)` is shape `(3, 1)`
- `b` is shape `(3, 4)`

**Broadcasting rule**: When dimensions don’t match, PyTorch inserts a `1` in the smaller dimension and stretches it as needed.

So effectively:
```
(3, 1) * (3, 4)
becomes
(3, 4) * (3, 4)
```

Each element of the column vector (from `a[i, :]`) is multiplied with the corresponding row in `b`.

#### 4. `.sum(dim=0)`
This sums along **rows**, resulting in a vector of shape `(4,)`.

So:
```python
c[i] = (3, 1) * (3, 4) → (3, 4) → sum over dim=0 → (4,)
```

This becomes the `i`th row of the result `c`.

---

### Visual Intuition

Say:
```python
a[i,:] = [a1, a2, a3]  → shape (3,)
b = [[b11, b12, b13, b14],
     [b21, b22, b23, b24],
     [b31, b32, b33, b34]]  → shape (3, 4)
```

Then `a[i,:].unsqueeze(-1) = [[a1], [a2], [a3]]` → shape (3, 1)

Now:
```python
a[i,:].unsqueeze(-1) * b =
[[a1*b11, a1*b12, a1*b13, a1*b14],
 [a2*b21, a2*b22, a2*b23, a2*b24],
 [a3*b31, a3*b32, a3*b33, a3*b34]]  → shape (3, 4)

.sum(dim=0) =
[ a1*b11 + a2*b21 + a3*b31,
  a1*b12 + a2*b22 + a3*b32,
  a1*b13 + a2*b23 + a3*b33,
  a1*b14 + a2*b24 + a3*b34 ]
```

Which is exactly the dot product of the `i`th row of `a` with each column of `b`.

---

### Summary

This line:
```python
c[i] = (a[i,:].unsqueeze(-1) * b).sum(dim=0)
```

is a clever use of broadcasting to compute one row of matrix multiplication at a time by:
- Turning the row of `a` into a column (`(ac,)` → `(ac, 1)`)
- Multiplying it with `b` of shape `(ac, bc)` to get a broadcasted result
- Summing along rows to compute the dot products

Let me know if you want the same logic visualized with real numbers or debug prints.

In [ ]:
#| exports

def matrix_multiply_torch_optimized(a: List[List[float]], # input matrix of size (m, n)
                                    b: List[List[float]] # input matrix of size (n, p)
                                    ) -> List[List[float]]: # output matrix of size (m, p)
        a = torch.tensor(a)
        b = torch.tensor(b)
        ar, ac = a.shape
        br, bc = b.shape
        if ac != br:
            raise ValueError("Incompatible shapes for matrix multiplication")
        result = torch.zeros((ar, bc))
        for i in range(ar):
              result[i] = (a[i, :].unsqueeze(1) * b).sum(dim = 0)
              print(result[i], a[i, :].unsqueeze(1), b)
        return result

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()